In [149]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

In [155]:
def process_images(folder, label, x, y):
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping unreadable image: {img_path}")
            continue
        img = cv2.resize(img, (64, 64))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        features = hog(
            gray,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            block_norm='L2-Hys'
        )
        
        x.append(features)
        y.append(label)


In [156]:
cat_folder = r"C:\Users\Muhammad\train\cat"
dog_folder = r"C:\Users\Muhammad\train\dog"

In [157]:
x = []
y = []

In [158]:
process_images(cat_folder, 1, x, y)  # Cat = 1
process_images(dog_folder, 0, x, y)  # Dog = 0

In [159]:
x = np.array(x)
y = np.array(y)

print(f"Total images loaded: {len(x)}")
print("Label distribution:", np.bincount(y))


Total images loaded: 176
Label distribution: [86 90]


In [160]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
print("Train label counts:", np.bincount(y_train))
print("Test label counts:", np.bincount(y_test))


Train label counts: [68 72]
Test label counts: [18 18]


In [161]:
model = SVC(kernel='rbf', gamma='scale')
model.fit(x_train, y_train)


SVC()

In [162]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 86.11%


In [163]:
def predict_image(img_path, model):
    print(f"Processing image: {img_path}")
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Could not read image {img_path}")
        return

    img = cv2.resize(img, (64, 64))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    features = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )
    features = features.reshape(1, -1)

    prediction = model.predict(features)
    label = "Dog" if prediction[0] == 0 else "Cat"
    print(f"Predicted: {label}")
   

In [164]:
predict_image(r"C:\Users\Muhammad\train\dog\american_pit_bull_terrier_182_jpg.rf.80b6c1327c7e638c1eaec176bdd0f40c.jpg",model)
predict_image(r"C:\Users\Muhammad\train\cat\Bengal_174_jpg.rf.bbac947432fbff310b6be201bd9a3de7.jpg",model)
predict_image(r"C:\Users\Muhammad\train\dog\beagle_112_jpg.rf.1eb40b396869a196e92a82f4a28c1d8b.jpg",model)
predict_image(r"C:\Users\Muhammad\train\cat\Birman_166_jpg.rf.ac8785ee575600493ffac45e2b0d1342.jpg",model)

Processing image: C:\Users\Muhammad\train\dog\american_pit_bull_terrier_182_jpg.rf.80b6c1327c7e638c1eaec176bdd0f40c.jpg
Predicted: Dog
Processing image: C:\Users\Muhammad\train\cat\Bengal_174_jpg.rf.bbac947432fbff310b6be201bd9a3de7.jpg
Predicted: Cat
Processing image: C:\Users\Muhammad\train\dog\beagle_112_jpg.rf.1eb40b396869a196e92a82f4a28c1d8b.jpg
Predicted: Dog
Processing image: C:\Users\Muhammad\train\cat\Birman_166_jpg.rf.ac8785ee575600493ffac45e2b0d1342.jpg
Predicted: Cat
